In [ ]:
import requests
import urllib.parse
import json
import re
from IPython.display import clear_output

In [ ]:
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0"}
symbol = 'کاردان'
ticker = urllib.parse.quote(symbol, encoding='utf-8')
url_1 = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=3&Consolidatable=true&IsNotAudited=false&Isic=46430613&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber=1&Publisher=false&Symbol={ticker}&TracingNo=-1&search=true'

response = requests.get(url_1, headers=headers)
pages = json.loads(response.text)['Page']

links_1 = []
for page in range(pages):
    page_url = f'https://search.codal.ir/api/search/v2/q?&Audited=true&AuditorRef=-1&Category=3&Childs=false&CompanyState=0&CompanyType=3&Consolidatable=true&IsNotAudited=false&Isic=46430613&Length=-1&LetterType=-1&Mains=true&NotAudited=true&NotConsolidatable=true&PageNumber={page+1}&Publisher=false&Symbol={ticker}&TracingNo=-1&search=true'
    response = requests.get(page_url, headers=headers)
    reports = json.loads(response.text)['Letters']
    for report in reports:
        AttachmentUrl = report['AttachmentUrl']
        Title = report['Title'].replace('/', '')
        TracingNo = report['TracingNo']
        links_1.append([AttachmentUrl, Title, TracingNo])

In [ ]:
pattern = r'DownloadFile\.aspx\?id='
failed_link1 = []

download_links = []
for j, link in enumerate(links_1):
    url = f'https://www.codal.ir{link[0]}'
    response = requests.get(url, headers=headers)
    page_source = response.text
    match = re.search(r'DownloadFile\.aspx\?id=[^&#]+', page_source)
    if match:
        download_id = match.group(0)
        download_link = f'https://www.codal.ir/Reports/{download_id}'
        download_links.append(download_link)
    else:
        print(f'failed: {url}')
        failed_link1.append(url)
        download_link = f'https://www.codal.ir/Reports/'
        download_links.append(download_link)

clear_output(wait=True)
print(f'failed: \n{failed_link1}')

In [ ]:
types = ['pdf', 'xlsx']
err = 'خطای سیستمی | کدال'
exl_type = '[Content_Types].xml'
failed = []
counter = 0
for i, link in zip(download_links, links_1):
    counter += 1
    if len(i) <= 29:
        clear_output(wait=True)
        print(f'error: {counter} of {len(download_links)} there is no file {symbol}_{link[1]}_{link[2]}')
        failed.append(f'{symbol}_{link[1]}_{link[2]}')
        continue
    else: 
        print(i)
        response = requests.get(i, headers=headers)
        if err in response.text[:100]:
            clear_output(wait=True)
            print(f'error: {counter} of {len(download_links)} there is no file {symbol}_{link[1]}_{link[2]}')
            failed.append(f'{symbol}_{link[1]}_{link[2]}')
            continue
        else:
            file_format = response.headers.get('content-disposition').split('=')[1].split('.')[-1][:-1]
            if file_format not in types:
                if exl_type in response.text[:100]:
                    file_format = 'xlsx'
                else:
                    file_format = 'pdf'
            with open(f'{symbol}_{link[1]}_{link[2]}.{file_format}', 'wb') as f:
                f.write(response.content)
            clear_output(wait=True)
            print(f'saved: {counter} of {len(download_links)} {symbol}_{link[1]}_{link[2]}.{file_format}')

